In [2]:
#
#Tweets Machine Learning Coursework 
#COMP3222
#

# To support both python 2 and 3
from __future__ import division, print_function, unicode_literals

# Common imports
import os
import numpy as np
import pandas as pd

# These two lines are required to use Tensorflow 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# To plot nice figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Clear tensorflow's and reset seed
def reset_graph(seed=None):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)



Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#get data
def getData(file_name):
    dict_data = {}
    data_file = open(file_name, "r", encoding="utf8")
    raw_data_txt = data_file.readlines()
    raw_data = []
    for data in raw_data_txt:
        raw_data.append(data.split("\t"))
    raw_data.pop(0)

    target_text = [data[6] for data in raw_data]
    dict_data['text'] = np.array([tweet[1] for tweet in raw_data])
    dict_data['imageIds'] = np.array([tweet[3] for tweet in raw_data])
    dict_data['timestamp'] = np.array([tweet[5] for tweet in raw_data])
    dict_data['label'] = np.array([1 if target=='real' or target =='real\n' else 0 for target in target_text])

    return dict_data

df_train = pd.DataFrame.from_dict(getData("mediaeval-2015-trainingset.txt"))
df_test = pd.DataFrame.from_dict(getData("mediaeval-2015-testset.txt"))
print("Done")


Done


In [36]:
#save file pre-processing to csv
#df_test.to_csv('test_v3.csv')

In [3]:
df_train = pd.read_csv("train_v2.2.csv")
df_train = df_train.iloc[:, 1:]
df_train
df_train.text = df_train.text.astype(str)

In [4]:
df_test = pd.read_csv("test_v2.2.csv")
df_test = df_test.iloc[:, 1:]
df_test
df_test.text = df_test.text.astype(str)

In [5]:
#Find all the different languages
#https://meta.wikimedia.org/wiki/Template:List_of_language_names_ordered_by_code conversion of symbol to langauge table
from langdetect import detect

def language_detect(data_frame):
    languages = []
    for tweet in data_frame['text'].values:
        try:
            languages.append(detect(tweet))
        except:
            languages.append('en')

    data_frame.insert(3, "language", languages, True)

    languages_seen = set(languages)
    print("Languages seen: ", languages_seen)
    return data_frame

In [6]:
def retweet_detection(data_frame):
    retweets = [1 if tweet.count("RT") >= 1 else 0 for tweet in data_frame["text"].values]
    data_frame.insert(4, "retweet", retweets, True)
    return data_frame

In [7]:
from deep_translator import GoogleTranslator

def translate(data_frame):
    translate_text = {}
    for i in range(len(data_frame)):
        if(data_frame.iloc[i]['language'] != 'en'):
            translate_text[data_frame.iloc[i]['text']] = GoogleTranslator(source='auto', target='en').translate(data_frame.iloc[i]['text'])
            #print(i, data_frame.iloc[i]['language'], GoogleTranslator(source='auto', target='en').translate(df_train.iloc[i]['text']))

    data_frame["text"].replace(translate_text, inplace=True)
    return data_frame

In [49]:
df_train

,text,pos,imageIds,timestamp,language,num_emoji,num_hashtags,profane,retweet,label
0,remember movie day tomorrow reminds happening ...,VB NN NN NN VBZ VBG NN NN,sandyA_fake_46,1351550041,es,0,1,0,0,0
1,look sandy ny tremendous image hurricane looks...,NN JJ RB JJ NN NN VBZ IN NN NN CD JJ NN,sandyA_fake_09,1351537883,es,0,0,0,1,0
2,good photo hurricane sandy reminds movie indep...,JJ NN NN JJ VBZ NN NN NN VBZ NN,sandyA_fake_09,1351534268,es,0,2,0,0,0
3,scary shit hurricane ny,JJ VBD NN NN,sandyA_fake_29,1351538133,en,0,2,1,0,0
4,fave place world nyc hurricane sandy statueofl...,JJ NN NN NN NN NN NN,sandyA_fake_15,1351543562,en,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...
14478,slaps tweetdeck pigfish http,NNS VBP JJ NN,pigFish_01,-1,en,0,0,0,0,0
14479,new species fish found brazil really good phot...,JJ NNS VBP VBN IN RB JJ NN VBP NN,pigFish_01,-1,en,0,0,0,0,0
14480,call pigfish http,VB JJ NN,pigFish_01,-1,en,0,1,0,0,0
14481,pigfish shark pork fish http co hqzwghydef,JJ NN NN JJ NN NN NN,pigFish_01,-1,it,0,0,0,0,0


In [9]:
import spacy
from profanity_filter import ProfanityFilter

def profanity_detection(data_frame):
    nlp = spacy.load('en_core_web_sm')
    profanity_filter = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
    nlp.add_pipe(profanity_filter.spacy_component, last=True)

    doc = nlp('This is fuck shit!')

    doc._.is_profane
    profanity_check = []
    count = 0
    for tweet in data_frame['text'].values:
        text = nlp(tweet)
        if(text._.is_profane):
            profanity_check.append(1)
        else: 
            profanity_check.append(0)
        
        #print(count, text._.is_profane)
        count += 1
    
    data_frame.insert(4, "profane", profanity_check, True)
    return data_frame

In [10]:
def numofhashtags(data_frame):
    num_hashtags = []
    for i in range(len(data_frame)):
        num_hashtags.append(data_frame.iloc[i]['text'].count("#"))
    #maybe count number of punctuation as well.
    data_frame.insert(4, "num_hashtags", num_hashtags, True)
    return data_frame


In [12]:
import emoji
def emoji_detect(data_frame):
    num_emojis = []
    for i in range(len(data_frame)):
        emoji_count = emoji.emoji_count(data_frame.iloc[i]['text'])
        num_emojis.append(emoji_count)
    
    data_frame.insert(4, "num_emoji", num_emojis, True)
    return data_frame


In [13]:
import regex as re

def parseText(data_frame):
    parsed_text = {}
    for i in range(len(data_frame)):
        parsed = ' '.join(re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",data_frame.iloc[i]['text']).split())
        parsed_text[data_frame.iloc[i]['text']] = parsed.lower()
        #print(i, parsed)
    data_frame["text"].replace(parsed_text, inplace=True)
    return data_frame
    

In [14]:
#based of https://craft.co/cnn/competitors
#
def new_companies(data_frame):
    news_companies =["cnn", "bbc", "nbc", "new york times", "wall street journal", " ap ", "fox", "cnbc", "daily mail"]
    news_found = []
    for tweet in data_frame['text'].values:
        tweet = tweet.lower()
        if(any(news in tweet for news in news_companies) or (tweet.find('ap') == 0) or (tweet.find('ap') == len(tweet)-2) ) :
            news_found.append(1)
        else:
            news_found.append(0)
    data_frame.insert(4, "news_company_found", news_found, True)
    return data_frame

In [15]:
#location detector too much noise picked up

from geopy.geocoders import Nominatim
def location_detector():
    geolocator = Nominatim(user_agent="COMP322-MLT")
    x = "My fave place in the world nyc hurricane sandy times sqaure"
    x = x.split(' ')
    for i in range(len(x)):
        try:
            location = geolocator.geocode(x[i])
            print(location.address)
        except:
            print("Notfound")

In [16]:
#Run hashtags if first time running on machine

from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

def stopWordRemoval(data_frame):
    stop_words = set(stopwords.words('english'))
    filtered_text = {}
    for i in range(len(data_frame)):
        word_tokens = word_tokenize(data_frame.iloc[i]['text'])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        new_sentence = " ".join(filtered_sentence)
        filtered_text[data_frame.iloc[i]['text']] = new_sentence
        #print(i, new_sentence)
    data_frame["text"].replace(filtered_text, inplace=True)
    return data_frame

In [16]:
from spellchecker import SpellChecker
def misspelling_detection(data_frame):
    spell = SpellChecker()
    misspelled_count = []
    for i in range(len(data_frame)):
        word_list = word_tokenize(data_frame.iloc[i]['text'])
        misspelled = spell.unknown(word_list)
        misspelled_count.append(len(misspelled))
        #print(i, len(misspelled), misspelled))
    data_frame.insert(4, "misspellings", misspelled_count, True)
    return data_frame

In [17]:
from collections import defaultdict, Counter

fake_words = defaultdict(lambda: 0)
real_words = defaultdict(lambda: 0)
total_words = defaultdict(lambda: 0)

for i in range(len(df_train)):
    text = str(df_train.iloc[i]['text']).lower()
    text = text.split()
    if(df_train.iloc[i]['label'] == 0):
        for j in range(len(text)):
            fake_words[text[j]] += 1
            total_words[text[j]] += 1
    else:
        for j in range(len(text)):
            real_words[text[j]] += 1
            total_words[text[j]] += 1


real_words = dict(sorted(real_words.items(), reverse=True, key=lambda item: item[1]))
fake_words = dict(sorted(fake_words.items(), reverse=True, key=lambda item: item[1]))
total_words = dict(sorted(total_words.items(), reverse=False, key=lambda item: item[1]))

words = list(total_words.keys())
for w in words:
    if(total_words[w] < 100):
        del total_words[w]
        try:
            del real_words[w]
        except:
            pass
        try:
            del fake_words[w]
        except:
            pass
    
rw = Counter(real_words)
fw = Counter(fake_words)
fake_res = fw - rw
real_res = rw - fw

real_res = {k: real_res[k]/total_words[k] for k in total_words.keys() & real_res}
fake_res = {k: fake_res[k]/total_words[k] for k in total_words.keys() & fake_res}

real_res = dict(sorted(real_res.items(), reverse=True, key=lambda item: item[1]))
fake_res = dict(sorted(fake_res.items(), reverse=True, key=lambda item: item[1]))


sig_real_words = list(k for k, v in real_res.items() if v > 0.5)
sig_fake_words = list(k for k, v in fake_res.items() if v > 0.5)

def significance(data_frame, sig_words, clmn_name):
    sig_found = []
    for tweet in data_frame['text'].values:
        try:
            tweet = tweet.lower()
            if(any(word in tweet for word in sig_words)):
                sig_found.append(1)
            else:
                sig_found.append(0)
        except:
            sig_found.append(0)
    data_frame.insert(4, clmn_name, sig_found, True)
    return data_frame

In [ ]:
from nltk import ngrams

def n_grams(data_frame, n):
    a = []
    for i in range(len(data_frame)):
        sentence = data_frame.iloc[i]['text']

        ng = ngrams(sentence.split(), n)
        x = []
        for grams in ng:
            x.append(grams)
        a.append(x)
    data_frame.insert(3, ng+"-gram", a, True)

In [55]:
df_test = language_detect(df_test)
print("Language Detection Done")
df_test = translate(df_test)
print("Translation Done")
df_test = retweet_detection(df_test)
print("Retweet Detection Done")
df_test = new_companies(df_test)
print("News companies Detection Done")
df_test = numofhashtags(df_test)
print("Hashtag count Done")
df_test = emoji_detect(df_test)
print("Emoji count Done")
df_test = parseText(df_test)
print("Parsing Done")
df_test = stopWordRemoval(df_test)
print("Stopwprd removal Done")
df_test = misspelling_detection(df_test)
print("Misspelling Count Done")
df_test = profanity_detection(df_test)
print("Profanity Detection Done")
df_test = significance(df_test, sig_real_words, "significant_real")
print("Real Significant Words Detected")
df_test = significance(df_test, sig_fake_words, "significant_fake")
print("Fake Significant Words Detected")

print("Preprocessing done")
#before passing the text as unigram .lower everything

Languages seen:  {'ta', 'sq', 'lt', 'el', 'ar', 'so', 'tl', 'pl', 'hi', 'de', 'ro', 'ru', 'th', 'fr', 'ko', 'es', 'ca', 'en', 'ja', 'pt', 'af', 'id', 'cy', 'nl', 'et', 'it', 'fi', 'sv', 'te', 'vi', 'tr', 'bg', 'hr'}
Language Detection Done
Translation Done
Retweet Detection Done
News companies Detection Done
Hashtag count Done
Emoji count Done
Parsing Done
Stopwprd removal Done
Misspelling Count Done
Profanity Detection Done
Preprocessing done


In [36]:
df_test

,text,pos,imageIds,timestamp,language,profane,misspellings,num_emoji,label
0,kereeen rt eclipse iss,JJ NN NN NN,eclipse_01,1426844743,en,0,2,0,0
1,absolutely beautiful rt eclipse iss,RB JJ NN NN NN,eclipse_01,1426849442,en,0,1,0,0
2,eclipse iss 3 20 wow amazing,NN VBZ CD CD NN NN,eclipse_01,1426853406,en,0,0,0,0
3,eclipse iss,NN NN,eclipse_01,1426842761,en,0,0,0,0
4,eclipse seen iss something else divine creatio...,NN VBN JJ NN RB JJ NN NN,eclipse_01,1426873451,fr,0,0,1,0
...,...,...,...,...,...,...,...,...,...
3776,zdf presenter confesses rigged video varoufaki...,NN NN NNS VBD NN NN NN JJ NN IN,varoufakis_1,1426744184,fr,0,2,0,0
3777,oh kleine liars zdf presenter confesses faked ...,UH NN NNS VBP NN NNS VBD JJ NN NN,varoufakis_1,1426744302,fr,0,2,0,0
3778,zdf program confirm varoufakis video montage,NN NN NN VBZ JJ NN,varoufakis_1,1426757001,es,0,2,0,0
3779,11 34 almost noon big confusion varoufakis vid...,CD CD RB RB JJ NN NN NN NN JJ NN NN NN,varoufakis_1,1426761320,de,0,4,2,0


In [46]:
df_train

,text,pos,imageIds,timestamp,language,num_emoji,num_hashtags,profane,retweet,label
0,[],VB NN NN NN VBZ VBG NN NN,sandyA_fake_46,1351550041,es,0,1,0,0,0
1,[],NN JJ RB JJ NN NN VBZ IN NN NN CD JJ NN,sandyA_fake_09,1351537883,es,0,0,0,1,0
2,[],JJ NN NN JJ VBZ NN NN NN VBZ NN,sandyA_fake_09,1351534268,es,0,2,0,0,0
3,[],JJ VBD NN NN,sandyA_fake_29,1351538133,en,0,2,1,0,0
4,[],JJ NN NN NN NN NN NN,sandyA_fake_15,1351543562,en,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...
14478,[],NNS VBP JJ NN,pigFish_01,-1,en,0,0,0,0,0
14479,[],JJ NNS VBP VBN IN RB JJ NN VBP NN,pigFish_01,-1,en,0,0,0,0,0
14480,[],VB JJ NN,pigFish_01,-1,en,0,1,0,0,0
14481,[],JJ NN NN JJ NN NN NN,pigFish_01,-1,it,0,0,0,0,0


In [19]:
df_train = df_train[['text', 'imageIds', 'timestamp', 'language', 'significant_fake', 'significant_real', 'profane', 'misspellings', 'num_emoji', 'num_hashtags', 'news_company_found', 'retweet', 'label']]
df_train

,text,imageIds,timestamp,language,significant_fake,significant_real,profane,misspellings,num_emoji,num_hashtags,news_company_found,retweet,label
0,remember movie Day Tomorrow reminds happening ...,sandyA_fake_46,Mon Oct 29 22:34:01 +0000 2012,es,1,0,0,0,0,1,0,0,0
1,Look Sandy NY Tremendous image hurricane Looks...,sandyA_fake_09,Mon Oct 29 19:11:23 +0000 2012,es,1,0,0,1,0,0,0,1,0
2,Good photo Hurricane Sandy reminds movie Indep...,sandyA_fake_09,Mon Oct 29 18:11:08 +0000 2012,es,1,0,0,1,0,2,0,0,0
3,Scary shit hurricane NY,sandyA_fake_29,Mon Oct 29 19:15:33 +0000 2012,en,1,0,1,0,0,2,0,0,0
4,fave place world nyc hurricane sandy statueofl...,sandyA_fake_15,Mon Oct 29 20:46:02 +0000 2012,en,1,0,0,2,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14478,slaps TweetDeck PigFish http co pyHcJn0jwA,pigFish_01,Tue Mar 11 03: 48: 36 +0000 2014,en,0,0,0,4,0,0,0,0,0
14479,New Species Fish found Brazil Really good Phot...,pigFish_01,Mon Mar 10 18: 09: 26 +0000 2014,en,1,0,0,2,0,0,0,0,0
14480,call pigFISH http co 4Bml62OD15,pigFish_01,Mon Mar 10 10: 59: 45 +0000 2014,en,0,0,0,3,0,1,0,0,0
14481,Pigfish shark pork fish http co hQzWGhyDef,pigFish_01,Sun Mar 09 20: 07: 10 +0000 2014,it,1,0,0,3,0,0,0,0,0


In [54]:
def pull_features(data_frame):
    features = []
    for i in range(len(data_frame)):
        temp = []
        #temp.append(data_frame.iloc[i]['significant_fake'])
        #temp.append(data_frame.iloc[i]['significant_real'])
        temp.append(data_frame.iloc[i]['profane'])
        #temp.append(data_frame.iloc[i]['misspellings'])
        #temp.append(data_frame.iloc[i]['num_emoji'])
        #temp.append(data_frame.iloc[i]['num_hashtags'])
        #temp.append(data_frame.iloc[i]['news_company_found'])
        features.append(temp)
    features = np.array(features)
    return features

In [5]:
texts = df_train['text']
texts.append(df_test['text'])
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
df_train['text'] = tokenizer.texts_to_sequences(df_train['text'])
MAX_TWEET_LENGTH = 16

df_test['text'] = tokenizer.texts_to_sequences(df_test['text'])

biggest_token = max(tokenizer.index_word)

In [6]:
X_train_text = df_train.pop('text')
X_test_text = df_test.pop('text')

X_train_text = tf.keras.preprocessing.sequence.pad_sequences(X_train_text, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')
X_test_text = tf.keras.preprocessing.sequence.pad_sequences(X_test_text, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')

In [55]:
X_train_features = pull_features(df_train)
X_test_features = pull_features(df_test)

In [15]:
X_train_features.max()

NameError: name 'X_train_features' is not defined

In [7]:
# feat_inputs = tf.keras.layers.Input(shape=(1,), name='features')
# feat_embed = tf.keras.layers.Embedding(10+1, 50, input_length=1, name='feature_embedding')(feat_inputs)
# feat_conv = tf.keras.layers.Conv1D(256, 1, name='feature_conv')(feat_embed)
# feat_pool = tf.keras.layers.GlobalMaxPool1D(name='feature_pooling')(feat_conv)


text_input = tf.keras.layers.Input(shape=(MAX_TWEET_LENGTH,), name='tweet_input')
text_embed = tf.keras.layers.Embedding(biggest_token + 1, 50, input_length=MAX_TWEET_LENGTH, name='article_body_embedding')(text_input)
text_conv = tf.keras.layers.Conv1D(128, 8, name='article_body_conv')(text_embed)
text_pool = tf.keras.layers.GlobalAveragePooling1D(name='article_body_pooling')(text_conv)

# concat = tf.keras.layers.concatenate([feat_pool, text_pool])
dense_10 = tf.keras.layers.Dense(75, activation='selu')(text_pool)
dense_5= tf.keras.layers.Dense(50, activation='selu')(dense_10)
dense_2= tf.keras.layers.Dense(20, activation='selu')(dense_5)
out_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_2)
model = tf.keras.models.Model(inputs=[text_input], outputs=out_layer)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tweet_input (InputLayer)     [(None, 16)]              0         
_________________________________________________________________
article_body_embedding (Embe (None, 16, 50)            454500    
_________________________________________________________________
article_body_conv (Conv1D)   (None, 9, 128)            51328     
_________________________________________________________________
article_body_pooling (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 75)                9675      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3800      
__

In [8]:
y_train = df_train['label'].values
y_test = df_test['label'].values
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='max', restore_best_weights=True)

history = model.fit([X_train_text], y_train, epochs=100, batch_size=128, validation_data=([X_test_text], y_test), callbacks=[callback])

Train on 14483 samples, validate on 3781 samples
Epoch 1/100
14483/14483 [==============================] - ETA: 0s - loss: 0.3578 - acc: 0.8385WARNING:tensorflow:From C:\Users\borin\anaconda3\envs\coursework\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
14483/14483 [==============================] - 3s 202us/sample - loss: 0.3578 - acc: 0.8385 - val_loss: 0.9139 - val_acc: 0.4806
Epoch 2/100
14483/14483 [==============================] - 3s 174us/sample - loss: 0.1698 - acc: 0.9338 - val_loss: 1.0724 - val_acc: 0.5073
Epoch 3/100
14483/14483 [==============================] - 3s 176us/sample - loss: 0.1183 - acc: 0.9559 - val_loss: 0.8845 - val_acc: 0.5988
Epoch 4/100
14483/14483 [==============================] - 3s 180us/sam

In [9]:
import sklearn.metrics as skl_m

preds = model.predict([X_test_text])
print(model.evaluate([X_test_text], y_test))

y_pred = []
for i in range(len(preds)):
    if(preds[i]<0.5):
        y_pred.append(0)
    else:
        y_pred.append(1)
    
y_pred = np.array(y_pred)

f1 = skl_m.f1_score(y_test, y_pred, average='micro')
f1

[0.7348672887401523, 0.8407829]


0.840782861676805